***
*Project:* Helmholtz Machine on Niche Construction

*Author:* Jingwei Liu, Computer Music Ph.D., UC San Diego
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Experiment 4_1</span> 

# Helmholtz Machine Test on Deep Structure

Instead of the toy model on well-formed set, we and more randomness to the dataset with Baysian mixture of Gaussians.

*Created:* December 24, 2023

*Updated:* December 24, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import utils as ut

<img src="muti-bernoulli.jpg" style="width:800px">
<caption><center> **Figure 2**: Multivariate Bernoulli Distribution. In wake phase, we go from input $\mathbf{x}$ to output $\mathbf{y}$ by weight $\Phi$. Blue neurons represent instantiation layers, where each neuron takes binary value and is computed as an independent Bernoulli variable. Orange neurons are inserted activations, which transform a shallow neural network with one-step prameter updating to deep neural network with backpropogation.
</center></caption>

Notations: 
- Bold lower-case math symbols represent column vectors
- Bold upper-case math symbols represent matrices
- “$\centerdot$” represents element-wise multiplication
- "$\times$" or "" represents matrix multiplication
- $\otimes$ represents outer product

$$
\begin{align}
\log Q &= \sum_{n} \frac{y_n-b}{a-b} \log [q_n] + \frac{a-y_n}{a-b} \log [1-q_n] \\
\frac{\partial \log Q}{\partial \mathbf{q}} &= \frac{\mathbf{y}-b}{a-b} \centerdot \frac{1}{\mathbf{q}} - \frac{a-\mathbf{y}}{a-b} \centerdot \frac{1}{1-\mathbf{q}} 
\end{align}
$$

Since

$$
\mathbf{q} = \sigma(\Phi^{l3,y}\times\mathbf{z^3})
$$

where $\Phi^{l3,y}$ is a matrix and $\sigma'(\centerdot) = \sigma(\centerdot)(1-\sigma(\centerdot))$，

$$
\frac{\partial \mathbf{q}}{\partial \sigma(\mathbf{\centerdot})} = \mathbf{q}\centerdot(1-\mathbf{q})
$$

Therefore,

$$
\frac{\partial \log Q}{\partial \sigma(\mathbf{\centerdot})} = \frac{\partial \log Q}{\partial \mathbf{q}} \centerdot \frac{\partial \mathbf{q}}{\partial \sigma(\mathbf{\centerdot})} = \frac{\mathbf{y}-b}{a-b} - \mathbf{q}
$$

The objective cross entropy is $L = -\log Q$,

$$
\frac{\partial L}{\partial \sigma(\mathbf{\centerdot})} = \mathbf{q} - \frac{\mathbf{y}-b}{a-b}
$$

Let's denote $\mathbf{q} - \frac{\mathbf{y}-b}{a-b}$ as $\mathbf{u}$, which in more general represents the **element-wise** product of all previous derivative computations $\frac{\partial L}{\partial \mathbf{q}}$ and the derivative of the activation function $\frac{\partial \mathbf{q}}{\partial g(\mathbf{\centerdot})}$ (times $c$) if the activation layer is scaled by $c$. The activation function can be replaced by $tanh(\centerdot)$, whose derivative is $tanh'(\centerdot) = 1-tanh^2(\centerdot)$.

$$
\begin{align}
\frac{\partial L}{\partial \Phi} &= \mathbf{u} \otimes \mathbf{z} \\
\frac{\partial L}{\partial \mathbf{z}} &= \Phi^T \mathbf{u}
\end{align}
$$

In [16]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

In [2]:
def multi_Bernoulli_update(x,y,parameter_set,lr,value_set,activation_type,bias):
    """
    Arguments:
    x -- input instantiation layer, numpy array of shape (n,1)
    y -- target instantiation layer, numpy array of shape (m,1)
    parameter_set -- parameters from x to y. Python dictionary of length l+1, l is the number of inserted layers. 
    The keys are ordered sequentially from layer x to y.
    lr -- learning rate, decimals
    value_set -- list or array [a,b], where a is the positive outcome and b is the negative outcome of a Bernoulli experiment
    activation_type -- we provide 2 choices of activation functions: tanh(x) and sigmoid(x)
    bias -- list or array [instantiation (data) bias, MLP bias], taking binary value in {True, False}. For example, 
    [False,True] means no instantiation bias but has MLP (data) bias
    
    Returns:
    parameter_set -- updated parameters
    loss -- value of loss function before updating, a number
    """
    
    inst_bias = bias[0]
    mlp_bias = bias[1]
    a = value_set[0]
    b = value_set[1]
    l = len(parameter_set)
    keys = [*parameter_set]
    G = {'z0': x}
    g = x
    
    for i in range(l-1):
        phi = parameter_set[keys[i]]
        if activation_type == "sigmoid":
            if mlp_bias == True:
                g = sigmoid(np.matmul(phi,np.append(g,[[1]], axis=0)))*(a-b)+b  # scale to [b,a]
            else:
                g = sigmoid(np.matmul(phi[:,:-1],g))*(a-b)+b
        elif activation_type == "tanh":
            if mlp_bias == True:
                g = np.tanh(np.matmul(phi,np.append(g,[[1]], axis=0)))*(a-b)/2+(a+b)/2 # scale to [b,a]
            else:
                g = np.tanh(np.matmul(phi[:,:-1],g))*(a-b)/2+(a+b)/2
        G['z'+str(i+1)] = g

    phi = parameter_set[keys[l-1]]
    if inst_bias == True:
        q = sigmoid(np.matmul(phi,np.append(g,[[1]], axis=0)))
    else:
        q = sigmoid(np.matmul(phi[:,:-1],g))
        
    # derivatives
    u = q - (y-b)/(a-b)
    loss = np.sum(np.abs(u))  # for visulization
    for i in range(l-1,0,-1):
        phi = parameter_set[keys[i]][:,:-1]
        dz = np.matmul(phi.T,u)
        z = G['z'+str(i)]
        parameter_set[keys[i]] -= lr * np.outer(u,np.append(z,[[1]], axis=0))
        if activation_type == "sigmoid":
            u = dz * z * (1-z) * (a-b)
        elif activation_type == "tanh":
            u = dz * (1-z**2) * (a-b)/2
            
    parameter_set[keys[0]] -= lr * np.outer(u,np.append(x,[[1]], axis=0))
    
    return parameter_set,loss

### Debug

In [3]:
n_y = 4
value_set = [1,-1]

In [4]:
y_set = ut.all_comb(n_y, value_set)
y_set

array([[ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
        -1.,  1., -1.],
       [ 1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,
         1., -1., -1.],
       [ 1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,
        -1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
        -1., -1., -1.]])

In [6]:
n_x = 8
x_set = ut.random_generate(3,n_x,2**n_y,value_set)
x_set

array([[ 1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1,  1,  1],
       [ 1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1],
       [ 1, -1,  1, -1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1, -1, -1],
       [ 1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1,  1],
       [-1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1],
       [-1, -1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1],
       [ 1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1, -1, -1,  1, -1,  1],
       [-1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1,  1]])

In [7]:
structure = [[8,4,1],
             [6,0,0],
             [5,0,0]]
n_dz = np.array(structure)
n_dz

array([[8, 4, 1],
       [6, 0, 0],
       [5, 0, 0]])

In [122]:
init_type = "random"
Phi, Theta = ut.parameter_initialization(init_type,n_dz)

In [123]:
Phi

{'Phi_01_1': array([[ 1.03600516, -1.4494836 , -0.04444709, -0.21871712, -0.19183653,
         -1.4078667 , -1.08134628,  1.22709755,  0.1136707 ],
        [ 0.59347517, -0.10852673,  0.92424733,  1.54242032,  0.7231468 ,
          0.21907053, -0.1525631 ,  0.06557071,  1.38991575],
        [-0.67111943,  0.70027428, -0.65541812, -1.16465332, -1.26014503,
          1.25329573,  0.06115812, -1.05875072,  0.96773782],
        [ 2.45908271, -0.65074003,  2.4798748 , -0.46928134, -0.16622484,
         -1.81304637, -0.48427655, -0.4585939 , -0.45546152],
        [-0.12193521, -0.90515638, -0.18679707, -1.25647995,  0.0433639 ,
          1.36869443, -0.20043603,  0.62497758,  0.93427925],
        [-0.00583617, -0.54156307, -0.75461135, -1.15417641, -0.14525051,
         -0.54347223,  1.12043731,  2.05341483,  0.76629276]]),
 'Phi_01_2': array([[ 0.43405654,  1.63621071, -1.00711939,  0.20765131, -0.10994907,
         -2.3777785 , -0.86381285],
        [ 1.6245528 ,  1.35600366,  1.06126593, 

In [124]:
parameter_set = {k: Phi[k] for k in ["Phi_" + str(0) + str(1) + "_" + str(j) for j in range(1,4)]}
parameter_set

{'Phi_01_1': array([[ 1.03600516, -1.4494836 , -0.04444709, -0.21871712, -0.19183653,
         -1.4078667 , -1.08134628,  1.22709755,  0.1136707 ],
        [ 0.59347517, -0.10852673,  0.92424733,  1.54242032,  0.7231468 ,
          0.21907053, -0.1525631 ,  0.06557071,  1.38991575],
        [-0.67111943,  0.70027428, -0.65541812, -1.16465332, -1.26014503,
          1.25329573,  0.06115812, -1.05875072,  0.96773782],
        [ 2.45908271, -0.65074003,  2.4798748 , -0.46928134, -0.16622484,
         -1.81304637, -0.48427655, -0.4585939 , -0.45546152],
        [-0.12193521, -0.90515638, -0.18679707, -1.25647995,  0.0433639 ,
          1.36869443, -0.20043603,  0.62497758,  0.93427925],
        [-0.00583617, -0.54156307, -0.75461135, -1.15417641, -0.14525051,
         -0.54347223,  1.12043731,  2.05341483,  0.76629276]]),
 'Phi_01_2': array([[ 0.43405654,  1.63621071, -1.00711939,  0.20765131, -0.10994907,
         -2.3777785 , -0.86381285],
        [ 1.6245528 ,  1.35600366,  1.06126593, 

In [138]:
lr = 0.05
activation_type = 'tanh'
bias = [True, True]

In [139]:
parameter_set,loss = multi_Bernoulli_update(x_set[:,2:3],y_set[:,2:3],parameter_set,lr,value_set,activation_type,bias)

In [140]:
parameter_set

{'Phi_01_1': array([[ 1.03412822, -1.44685152, -0.04181501, -0.22059406, -0.19371347,
         -1.40523463, -1.08397835,  1.22446547,  0.11179376],
        [ 0.55498396, -0.07003808,  0.96273598,  1.50392911,  0.68465559,
          0.25755917, -0.19105174,  0.02708206,  1.35142454],
        [-0.64770932,  0.72368753, -0.63200486, -1.14124321, -1.23673491,
          1.27670899,  0.03774486, -1.08216398,  0.99114793],
        [ 2.61151027, -0.80555735,  2.32505748, -0.31685379, -0.01379728,
         -1.96786369, -0.32945923, -0.30377658, -0.30303397],
        [-0.12730666, -0.90768393, -0.18932461, -1.26185141,  0.03799245,
          1.36616689, -0.19790849,  0.62750512,  0.9289078 ],
        [-0.00441149, -0.54299024, -0.75603852, -1.15275173, -0.14382582,
         -0.5448994 ,  1.12186447,  2.05484199,  0.76771744]]),
 'Phi_01_2': array([[ 1.27599386e-01,  1.35363115e+00, -6.99961185e-01,
         -8.53874352e-02,  1.40069870e-01, -2.68426834e+00,
         -1.17142371e+00],
        [ 1

In [141]:
loss

1.832358851943358

In [150]:
for i in range(10):
    parameter_set,loss = multi_Bernoulli_update(x_set[:,2:3],y_set[:,2:3],parameter_set,lr,value_set,activation_type,bias)
    print(loss)

0.11482468038672713
0.11385079074403776
0.11289317666926531
0.1119514360514471
0.111025179836501
0.11011403150474905
0.10921762657318702
0.10833561212114713
0.10746764633808939
0.10661339809233239


In [158]:
z1 = np.tanh(np.matmul(parameter_set['Phi_01_1'],np.append(x_set[:,2:3],[[1]], axis=0)))
z1

array([[-0.96514144],
       [ 0.99996008],
       [ 0.96935636],
       [ 0.94893083],
       [-0.27967865],
       [-0.99997004]])

In [159]:
z2 = np.tanh(np.matmul(parameter_set['Phi_01_2'],np.append(z1,[[1]], axis=0)))
z2

array([[ 0.97043077],
       [ 0.93691366],
       [ 0.99983195],
       [ 0.8304695 ],
       [-0.9841447 ]])

In [160]:
y = sigmoid(np.matmul(parameter_set['Phi_01_3'],np.append(z2,[[1]], axis=0)))
y

array([[0.97399504],
       [0.02226595],
       [0.97908443],
       [0.96341393]])

In [161]:
y_set[:,2:3]

array([[ 1.],
       [-1.],
       [ 1.],
       [ 1.]])

In [162]:
parameter_set

{'Phi_01_1': array([[ 1.06573372, -1.41478322, -0.00974671, -0.18898856, -0.16210797,
         -1.37316633, -1.11604665,  1.19239717,  0.14339926],
        [ 0.5535548 , -0.06856137,  0.96421269,  1.50249995,  0.68322643,
          0.25903589, -0.19252846,  0.02560535,  1.34999538],
        [-0.4585309 ,  0.91299425, -0.44269815, -0.95206479, -1.04755649,
          1.4660157 , -0.15156185, -1.27147069,  1.18032635],
        [ 2.5885674 , -0.85555704,  2.2750578 , -0.33979666, -0.03674015,
         -2.01786338, -0.27945954, -0.2537769 , -0.32597684],
        [-0.18425157, -0.80305546, -0.08469615, -1.31879631, -0.01895246,
          1.47079535, -0.30253695,  0.52287666,  0.87196289],
        [-0.00444796, -0.54304856, -0.75609685, -1.1527882 , -0.1438623 ,
         -0.54495772,  1.1219228 ,  2.05490032,  0.76768096]]),
 'Phi_01_2': array([[ 9.60050537e-02,  1.37463868e+00, -6.68313921e-01,
         -6.54621095e-02,  1.37786036e-01, -2.71674185e+00,
         -1.15046003e+00],
        [ 8

In [164]:
structure = [[10,8,6,3,1],
             [9, 7,5,2,0],
             [0, 0,0,0,0]]
n_dz = np.array(structure)
n_dz

array([[10,  8,  6,  3,  1],
       [ 9,  7,  5,  2,  0],
       [ 0,  0,  0,  0,  0]])

In [166]:
Phi, Theta = ut.parameter_initialization("zero",n_dz)  # "zero" or "random"

In [167]:
value_set = [1,0] # vanilla Helmholtz machine takes value {0,1}
activation_type = "sigmoid" # doesn't matter since vanilla Helmholtz machine doesn't have hidden MLP
bias = [False,False,False] # vanilla Helmholtz machine has no bias term

In [169]:
n = n_dz[0,0]
well_formed_set = ut.well_formed_generate(n,value_set)
well_formed_set

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 0., 1.],
       [1., 1., 0., ..., 1., 1., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [387]:
def reorder_all_comb(entire_set,dataset):
    """
    This function reorders the entire set with respect to the generated (or given) dataset. Since we are dealing with 
    categorical distributions, to visualize the result better, we put the datapoints contained in the dataset 
    (let's say k datapoints) to the first k columns of the entire_set, which are followed by false instances in subsequent
    columns.
        
    Arguments:
    entire_set -- a set containing all possible datapoints the input could be, numpy array of shape (n,2^n), 
    2^n is the number of all possible combinations of n binary neurons
    dataset -- generated (or given) dataset, numpy array of shape (n,n_data), n_data is the number of datapoints.
    
    Returns:
    reordered_set -- entire_set reordered as columns 0-k represents valid instances contained in the dataset, 
    columns k-2^n represents false instances not in the dataset. numpy array of shape (n,2^n)
    k -- number of distinct datapoints in dataset, integer
    """
    
    values,counts = np.unique(dataset, axis=1, return_counts = True)
    reordered_set = np.zeros(entire_set.shape)
    
    order = np.append(np.argsort(counts)[:int(counts.size/2)],np.flip(np.argsort(counts)[int(counts.size/2):]))
    dataset_arranged = values[:,order]

    k = counts.size
    reordered_set[:,:k] = dataset_arranged
    r = k
    for i in range(entire_set.shape[1]):
        flag = 0
        for j in range(k):
            if np.array_equal(entire_set[:,i], dataset[:,j]):
                flag = 1
                break
        if flag == 0:
            reordered_set[:,r] = entire_set[:,i]
            r += 1
    return reordered_set

In [388]:
reordered_set = reorder_all_comb(entire_set,dataset)

In [389]:
def metrics(generation,reordered_set,dataset):
    """
    Arguments:
    generation -- generated instances after training, numpy array of shape (n,n_sample), n is the length of input layer, 
    n_sample is the number of datapoints generated
    reordered_set -- entire_set reordered as columns 0-k represents valid instances contained in the dataset, 
    columns k-2^n represents false instances not in the dataset. numpy array of shape (n,2^n)
    dataset -- numpy array of shape (n,n_data), n_data is the number of datapoints.
    
    Returns:
    distribution -- assigned category for generated samples based on reordered set, numpy array of shape (n_sample, )
    data_dist -- assigned category for dataset  based on reordered set, numpy array of shape (n_data, )
    statistics -- python dictionary with keys:
        percent -- percentage of positive instances
        n_fn -- number of false negative samples, missing evidence
        FN -- position of false negative samples, numpy array of shape (k-n_fn, )
        n_fp -- number of false positive samples, outliers
        FP -- position and counts of false positive samples, numpy array of shape (2,n_fp)
    MSE -- mean squared error between the generation Q and the data evidence P on the support of P (on positive instances only).
    """
    n_sample = generation.shape[1]
    n_data = dataset.shape[1]
    distribution = np.zeros((n_sample, ),dtype = int)
    for i in range(n_sample):
        for j in range(reordered_set.shape[1]):
            if np.array_equal(generation[:,i], reordered_set[:,j]):
                distribution[i] = j
                break
    values_t, counts_t = np.unique(distribution, return_counts=True)
    
    data_dist = np.zeros((n_data, ),dtype = int)
    for i in range(n_data):
        for j in range(reordered_set.shape[1]):
            if np.array_equal(dataset[:,i], reordered_set[:,j]):
                data_dist[i] = j
                break
    values_d, counts_d  = np.unique(data_dist, return_counts=True)
    k = counts_d.size
    
    
    # statistics
    percent = np.sum(counts_t[values_t < k])/n_sample
    n_fn = k-values_t[values_t < k].size
    FN = np.zeros((n_fn,),dtype = int)
    dist_positive = np.array([values_t[values_t < k], counts_t[values_t < k]])
    s = 0
    values_t[values_t < k]
    dist_values = np.append(np.append(-1,values_t[values_t < k]),k)   # append 0 and k in the range
    
    for i in range(dist_values.size-1):
        diff = dist_values[i+1] - dist_values[i]
        for j in range(1,diff):
            FN[s] = dist_values[i]+j
            dist_positive = np.append(dist_positive, np.array([[dist_values[i]+j],[0]]),axis = 1)
            s += 1
    dist_positive = np.unique(dist_positive,axis = 1)
    n_fp = values_t[values_t >= k].size
    FP = np.array([values_t[values_t >= k], counts_t[values_t >= k]])
    statistics = {'percent': percent, 'FN': FN, 'n_fn':n_fn, 'FP': FP, 'n_fp':n_fp}
    
    # metric 2: distribution difference. Since our ditributions are discrete, we calculate a mean squared error (MSE) between 
    #           the generation Q and the data evidence P on the support of P (on positive instances only).
    
    counts_t = counts_t/n_sample*n_data  # distribution in the same scale as dataset
    MSE = np.sum((dist_positive[1,:]/n_sample*n_data - counts_d)**2)/k
    ABS_Error = np.abs(dist_positive[1,:]/n_sample*n_data - counts_d).sum()/k
    
    return distribution,data_dist,statistics, MSE,ABS_Error

In [391]:
distribution,data_dist,statistics, MSE,ABS_Error = metrics(entire_set,reordered_set,dataset)

In [368]:
n = 10
k = 5
n_data = 1000
random_set = ut.random_generate(k,n,n_data,value_set)
random_set

array([[1, 1, 0, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 1, 1]])

In [372]:
values,counts = np.unique(random_set, axis=1, return_counts = True)
counts.size

583

In [378]:
values

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 1, 1, ..., 1, 1, 0]])

In [382]:
int(counts.size/2)

291

In [383]:
np.argsort(counts)[:int(counts.size/2)]

array([582, 461, 254, 462, 252, 463, 250, 249, 248, 256, 247, 465, 244,
       243, 242, 241, 240, 239, 238, 246, 460, 458, 260, 282, 281, 444,
       279, 446, 447, 276, 448, 274, 273, 272, 271, 450, 451, 268, 454,
       264, 457, 261, 466, 283, 236, 233, 207, 206, 484, 485, 203, 201,
       200, 487, 210, 490, 389, 194, 492, 493, 190, 187, 186, 185, 196,
       480, 212, 213, 232, 231, 230, 229, 228, 470, 471, 225, 224, 472,
       473, 474, 475, 476, 477, 217, 478, 215, 479, 468, 184, 442, 287,
       359, 407, 357, 355, 409, 353, 410, 411, 360, 350, 412, 347, 345,
       415, 417, 419, 340, 420, 349, 405, 364, 366, 391, 387, 386, 385,
       384, 383, 382, 381, 380, 377, 376, 374, 373, 395, 396, 397, 369,
       398, 399, 422, 285, 337, 334, 307, 306, 305, 304, 303, 302, 301,
       299, 308, 435, 296, 294, 293, 292, 438, 439, 289, 288, 436, 433,
       310, 311, 425, 427, 331, 330, 328, 327, 430, 324, 432, 321, 320,
       319, 318, 317, 316, 315, 314, 313, 312, 335, 179, 491,  8

In [384]:
np.flip(np.argsort(counts)[int(counts.size/2)]:)

array([145, 137, 138, 139, 141, 142, 519, 144, 102, 155, 147, 148, 149,
       518, 151, 153, 516, 103, 390, 101,  36,  25,  26, 577, 574,  31,
        32,  33,  34,  35,  37, 100,  38,  40,  41,  42, 571, 569,  47,
       568, 567,  24, 578,  22,  21,   2,   3,   4,   5,   6,   7,   8,
         9, 579,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  50,
        51, 566,  78,  80,  81, 554,  83,  85,  86,  87, 178,  89,  90,
       552,  92,  93,  94,  95,  96,  97, 551,  99,  79,  77,  53,  76,
       565,  55,  57, 563,  59, 561,  61,  62,  64,  65, 557,  68, 556,
        70, 555,  72,  73,  74,  75,  88, 491, 179, 335, 312, 313, 314,
       315, 316, 317, 318, 319, 320, 321, 432, 324, 430, 327, 328, 330,
       331, 427, 425, 311, 310, 433, 436, 288, 289, 439, 438, 292, 293,
       294, 296, 435, 308, 299, 301, 302, 303, 304, 305, 306, 307, 334,
       337, 285, 422, 399, 398, 369, 397, 396, 395, 373, 374, 376, 377,
       380, 381, 382, 383, 384, 385, 386, 387, 391, 366, 364, 40

In [379]:
values[:,np.argsort(counts)]

array([[1, 1, 0, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 1, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 1, 1, 1]])

In [171]:
dataset = well_formed_set # well_formed_set or random_set
entire_set = ut.all_comb(n, value_set)
reordered_set = ut.reorder_all_comb(entire_set,dataset)
reordered_set

array([[1., 1., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [172]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

In [173]:
def wake_sample(n_dz,d0,value_set,Phi,activation_type,bias):
    """
    Arguments:
    n_dz -- number of neurons for each layer, numpy array of shape (n+1,m), where m is the number of instantiation layers, 
    n is the maximum number of inserted layers between adjacent instantiation layers
    d0 -- input pattern, numpy array of shape (n_d, 1)
    value_set -- list or array [a,b], where a is the positive outcome and b is the negative outcome of a Bernoulli experiment
    Phi -- Recognition parameter set, Python dictionary of length m-1 with each key-value pair being a parameter matrix of 
    shape (n_z{i+1}, n_zi+1), where the last column represents bias b's
    activation_type -- we provide 2 choices of activation functions: tanh(x) and sigmoid(x)
    bias -- list or array [instantiation bias, MLP bias], taking binary value in {True, False}. For example, [False,True] means 
    no instantiation bias but has MLP bias
    
    Returns:
    Alpha_Q -- assignment of each neuron (binary value), Python dictionary of length m-1 with each key-value pair being 
    a numpy array of shape (n_dz[0,i], 1),i = 0,...m-1
    """
    
    m = n_dz.shape[1]
    S = d0  # assignment of each layer
    Alpha_Q = {"z0":d0}
    inst_bias = bias[0]
    mlp_bias = bias[1]
    a = value_set[0]
    b = value_set[1]
    
    for i in range(m-2):
        n = np.where(n_dz[1:,i] != 0)[0].size  # number of inserted layers between i and i+1
        if n == 0:
            phi = Phi["Phi_" + str(i) + str(i+1)]
            if inst_bias == True:
                q = sigmoid(np.matmul(phi,np.append(S,[[1]], axis=0)))
            else:
                q = sigmoid(np.matmul(phi[:,:-1],S))
            S = ((q > np.random.rand(len(q),1)).astype(int))*(a-b)+b   # rejection sampling as a or b
            Alpha_Q["z"+str(i+1)] = S
        else:
            g = S
            for j in range(1,n+1):
                phi = Phi["Phi_" + str(i) + str(i+1) + "_" + str(j)]
                if activation_type == "sigmoid":
                    if mlp_bias == True:
                        g = sigmoid(np.matmul(phi,np.append(g,[[1]], axis=0)))*(a-b)+b  # scale to [b,a]
                    else:
                        g = sigmoid(np.matmul(phi[:,:-1],g))*(a-b)+b
                elif activation_type == "tanh":
                    if mlp_bias == True:
                        g = np.tanh(np.matmul(phi,np.append(g,[[1]], axis=0)))*(a-b)/2+(a+b)/2 # scale to [b,a]
                    else:
                        g = np.tanh(np.matmul(phi[:,:-1],g))*(a-b)/2+(a+b)/2
                    
            phi = Phi["Phi_" + str(i) + str(i+1) + "_" + str(j+1)]
            if inst_bias == True:
                q = sigmoid(np.matmul(phi,np.append(g,[[1]], axis=0)))
            else:
                q = sigmoid(np.matmul(phi[:,:-1],g))
            S = ((q > np.random.rand(len(q),1)).astype(int))*(a-b)+b
            Alpha_Q["z"+str(i+1)] = S
    Alpha_Q["z"+str(m-1)] = [[1]]
        
    return Alpha_Q

In [174]:
def sleep_sample(n_dz,value_set,Theta,activation_type,bias):
    """
    Arguments:
    n_dz -- number of neurons for each layer, numpy array of shape (n+1,m), where m is the number of instantiation layers, 
    n is the maximum number of inserted layers between adjacent instantiation layers
    value_set -- list or array [a,b], where a is the positive outcome and b is the negative outcome of a Bernoulli experiment
    Theta -- Generative parameter set, Python dictionary of length m with each key-value pair being a parameter matrix of 
    shape (n_z{i}, n_z{i+1}+1), where the last column represents bias b's
    activation_type -- we provide 2 choices of activation functions: tanh(x) and sigmoid(x)
    bias -- list or array [instantiation bias, MLP bias,data bias], taking binary value in {True, False}. For example, 
    [False,True,True] means no instantiation bias but has MLP bias and data bias
    
    Returns:
    Alpha_P -- assignment of each neuron (binary value), Python dictionary of length m-1 with each key-value pair being 
    a numpy array of shape (n_dz[0,i], 1),i = m-1,...,0
    """
    m = n_dz.shape[1]
    inst_bias = bias[0]
    mlp_bias = bias[1]
    data_bias = bias[2]
    a = value_set[0]
    b = value_set[1]
    S = [[1]]
    Alpha_P = {"z"+str(m-1):S}
    
    
    for i in range(m-1,0,-1):
        n = np.where(n_dz[1:,i-1] != 0)[0].size  # number of inserted layers between i and i-1
        if n == 0:
            theta = Theta["Theta_" + str(i) + str(i-1)]
            if i > 1:
                if inst_bias == True:
                    p = sigmoid(np.matmul(theta,np.append(S,[[1]], axis=0))) #
                else:
                    p = sigmoid(np.matmul(theta[:,:-1],S))
                S = ((p > np.random.rand(len(p),1)).astype(int))*(a-b)+b   # rejection sampling as a or b
                Alpha_P["z"+str(i-1)] = S
            else:
                if data_bias == True:
                    p = sigmoid(np.matmul(theta,np.append(S,[[1]], axis=0)))
                else:
                    p = sigmoid(np.matmul(theta[:,:-1],S))
                S = ((p > np.random.rand(len(p),1)).astype(int))*(a-b)+b   # rejection sampling as a or b
                Alpha_P["z"+str(i-1)] = S
        else:
            g = S
            for j in range(n+1,1,-1):
                theta = Theta["Theta_" + str(i) + str(i-1) + "_" + str(j)]
                if activation_type == "sigmoid":
                    if mlp_bias == True:
                        g = sigmoid(np.matmul(theta,np.append(g,[[1]], axis=0)))*(a-b)+b  # scale to [b,a]
                    else:
                        g = sigmoid(np.matmul(theta[:,:-1],g))*(a-b)+b
                elif activation_type == "tanh":
                    if mlp_bias == True:
                        g = np.tanh(np.matmul(theta,np.append(g,[[1]], axis=0)))*(a-b)/2+(a+b)/2 # scale to [b,a]
                    else:
                        g = np.tanh(np.matmul(theta[:,:-1],g))*(a-b)/2+(a+b)/2
                    
            theta = Theta["Theta_" + str(i) + str(i-1) + "_" + str(j-1)]
            
            if i > 1:
                if inst_bias == True:
                    p = sigmoid(np.matmul(theta,np.append(g,[[1]], axis=0)))
                else:
                    p = sigmoid(np.matmul(theta[:,:-1],g))
                S = ((p > np.random.rand(len(p),1)).astype(int))*(a-b)+b   # rejection sampling as a or b
                Alpha_P["z"+str(i-1)] = S
            else:
                if data_bias == True:
                    p = sigmoid(np.matmul(theta,np.append(g,[[1]], axis=0)))
                else:
                    p = sigmoid(np.matmul(theta[:,:-1],g))
                S = ((p > np.random.rand(len(p),1)).astype(int))*(a-b)+b   # rejection sampling as a or b
                Alpha_P["z"+str(i-1)] = S
            
    return Alpha_P

In [222]:
import logging

In [359]:
def multi_Bernoulli_update(x,y,parameter_set,lr,value_set,activation_type,bias):
    """
    Arguments:
    x -- input instantiation layer, numpy array of shape (n,1)
    y -- target instantiation layer, numpy array of shape (m,1)
    parameter_set -- parameters from x to y. Python dictionary of length l+1, l is the number of inserted layers. 
    The keys are ordered sequentially from layer x to y.
    lr -- learning rate, decimals
    value_set -- list or array [a,b], where a is the positive outcome and b is the negative outcome of a Bernoulli experiment
    activation_type -- we provide 2 choices of activation functions: tanh(x) and sigmoid(x)
    bias -- list or array [instantiation (data) bias, MLP bias], taking binary value in {True, False}. For example, 
    [False,True] means no instantiation bias but has MLP (data) bias
    
    Returns:
    parameter_set -- updated parameters
    loss -- value of loss function before updating, a number
    """
    
    inst_bias = bias[0]
    mlp_bias = bias[1]
    a = value_set[0]
    b = value_set[1]
    l = len(parameter_set)
    keys = [*parameter_set]
    G = {'z0': x}
    g = x
    
    for i in range(l-1):
        phi = parameter_set[keys[i]]
        if activation_type == "sigmoid":
            if mlp_bias == True:
                g = sigmoid(np.matmul(phi,np.append(g,[[1]], axis=0)))*(a-b)+b  # scale to [b,a]
            else:
                g = sigmoid(np.matmul(phi[:,:-1],g))*(a-b)+b
        elif activation_type == "tanh":
            if mlp_bias == True:
                g = np.tanh(np.matmul(phi,np.append(g,[[1]], axis=0)))*(a-b)/2+(a+b)/2 # scale to [b,a]
            else:
                g = np.tanh(np.matmul(phi[:,:-1],g))*(a-b)/2+(a+b)/2
        G['z'+str(i+1)] = g

    phi = parameter_set[keys[l-1]]
    if inst_bias == True:
        q = sigmoid(np.matmul(phi,np.append(g,[[1]], axis=0)))
    else:
        q = sigmoid(np.matmul(phi[:,:-1],g))
        
    # derivatives
    u = q - (y-b)/(a-b)
    loss = np.sum(np.abs(u))  # for visulization
    for i in range(l-1,0,-1):
        phi = parameter_set[keys[i]][:,:-1]
        dz = np.matmul(phi.T,u)
        z = G['z'+str(i)]
        parameter_set[keys[i]] -= lr * np.outer(u,np.append(z,[[1]], axis=0))
        if activation_type == "sigmoid":
            u = dz * z * (1-z) * (a-b)
        elif activation_type == "tanh":
            u = dz * (1-z**2) * (a-b)/2
            
    parameter_set[keys[0]] -= lr * np.outer(u,np.append(x,[[1]], axis=0))
    print(u)
    return parameter_set,loss,G

In [360]:
keys = [*parameter_set]
keys

['Theta_10_2', 'Theta_10_1']

In [361]:
def wake_update_delta(Phi,Alpha_P,lr,n_dz,value_set,activation_type,bias):
    """
    Arguments:
    Phi -- Recognition parameter set, Python dictionary of length m-1 with each key-value pair being a parameter matrix of 
    shape (n_z{i+1}, n_zi+1), where the last column represents bias b's
    Alpha_P -- assignment of each neuron (binary value), Python dictionary of length m with each key-value pair being 
    a numpy array of shape (n_dz[0,i], 1),i = m-1,...,0
    lr -- learning rate, decimals
    
    n_dz -- number of neurons for each layer, numpy array of shape (n+1,m), where m is the number of instantiation layers, 
    n is the maximum number of inserted layers between adjacent instantiation layers
    value_set -- list or array [a,b], where a is the positive outcome and b is the negative outcome of a Bernoulli experiment
    activation_type -- we provide 2 choices of activation functions: tanh(x) and sigmoid(x)
    bias -- list or array [instantiation bias, MLP bias], taking binary value in {True, False}. For example, [False,True] means 
    no instantiation bias but has MLP bias
    
    Returns:
    Phi -- Updated recognition parameter set, Python dictionary of length m-1 with each key-value pair being a parameter matrix of 
    shape (n_z{i+1}, n_zi+1), where the last column represents bias b's
    Loss -- numpy array of length m-1; the first m-2 values are layer loss, the last term is the total loss
    """
    m = n_dz.shape[1]
    Loss = np.zeros(m)
    for i in range(m-2):
        n = np.where(n_dz[1:,i] != 0)[0].size  # number of inserted layers between i and i+1
        if n == 0:
            parameter_set = {"Phi_" + str(i) + str(i+1): Phi["Phi_" + str(i) + str(i+1)]}
        else:
            parameter_set = {k: Phi[k] for k in ["Phi_" + str(i) + str(i+1) + "_" + str(j) for j in range(1,n+2)]}
            
        x = Alpha_P['z'+str(i)]
        y = Alpha_P['z'+str(i+1)]
        parameter_set,loss,G = multi_Bernoulli_update(x,y,parameter_set,lr,value_set,activation_type,bias)
        Loss[i] = loss
        Loss[-1] += loss
        for k in [*parameter_set]:
            Phi[k] = parameter_set[k]
        
    return Phi,Loss,G,parameter_set

In [362]:
def sleep_update_delta(Theta,Alpha_Q,lr,n_dz,value_set,activation_type,bias):
    """
    Arguments:
    Theta -- Generative parameter set, Python dictionary of length m with each key-value pair being a parameter matrix of 
    shape (n_z{i}, n_z{i+1}+1), where the last column represents bias b's
    Alpha_Q -- Recognition assignment of each neuron (binary value), Python dictionary of length m with each key-value pair being 
    a numpy array of shape (n_z, 1)
    lr -- learning rate, decimals
    
    n_dz -- number of neurons for each layer, numpy array of shape (n+1,m), where m is the number of instantiation layers, 
    n is the maximum number of inserted layers between adjacent instantiation layers
    value_set -- list or array [a,b], where a is the positive outcome and b is the negative outcome of a Bernoulli experiment
    activation_type -- we provide 2 choices of activation functions: tanh(x) and sigmoid(x)
    bias -- list or array [instantiation bias, MLP bias], taking binary value in {True, False}. For example, [False,True] means 
    no instantiation bias but has MLP bias
    
    Returns:
    Theta -- Updated generative parameter set, Python dictionary of length m with each key-value pair being a parameter matrix of 
    shape (n_z{i}, n_z{i+1}+1), where the last column represents bias b's
    Loss -- numpy array of length m; the first m-1 values are layer loss, the last term is the total loss
    """
    
    m = n_dz.shape[1]
    inst_bias = bias[0]
    mlp_bias = bias[1]
    data_bias = bias[2]
    
    Loss = np.zeros(m)
    bias = [inst_bias,mlp_bias]
    for i in range(m-1,1,-1):
        n = np.where(n_dz[1:,i-1] != 0)[0].size  # number of inserted layers between i and i+1
        if n == 0:
            parameter_set = {"Theta_" + str(i) + str(i-1): Theta["Theta_" + str(i) + str(i-1)]}
        else:
            parameter_set = {k: Theta[k] for k in ["Theta_" + str(i) + str(i-1) + "_" + str(j) for j in range(n+1,0,-1)]}
       
        x = Alpha_Q['z'+str(i)]
        y = Alpha_Q['z'+str(i-1)]
        parameter_set,loss,G = multi_Bernoulli_update(x,y,parameter_set,lr,value_set,activation_type,bias)
        Loss[i-1] = loss
        Loss[-1] += loss
        for k in [*parameter_set]:
            Theta[k] = parameter_set[k]
            
    bias = [data_bias,mlp_bias]     
    n = np.where(n_dz[1:,0] != 0)[0].size  # number of inserted layers between 0 and 1
    if n == 0:
        parameter_set = {"Theta_10": Theta["Theta_10"]}
    else:
        parameter_set = {k: Theta[k] for k in ["Theta_10_"+ str(j) for j in range(n+1,0,-1)]}

    x = Alpha_Q['z1']
    y = Alpha_Q['z0']
    parameter_set,loss,G = multi_Bernoulli_update(x,y,parameter_set,lr,value_set,activation_type,bias)
    Loss[0] = loss
    Loss[-1] += loss
    for k in [*parameter_set]:
        Theta[k] = parameter_set[k]

    return Theta,Loss,G,parameter_set

In [363]:
n_data = dataset.shape[1]
n_layer = n_dz.shape[1]

In [364]:
Phi, Theta = ut.parameter_initialization("random",n_dz)  # "zero" or "random"
Theta

{'Theta_10_1': array([[ 0.6010653 , -0.48508052,  0.6774026 , -0.82140179,  0.937665  ,
          2.29049904,  0.57965442,  0.7413375 ,  0.98325658, -1.60315525],
        [-0.81068888,  1.07310774,  0.99413419, -0.80715707, -0.73535167,
         -1.27548043, -0.68263782, -0.32889438, -1.03038359, -1.08745944],
        [-0.8054341 , -1.29747267, -0.52761607, -1.17408012,  0.4406551 ,
          0.26973603,  0.7454193 ,  1.2352713 ,  0.21740866,  0.36839592],
        [-1.34705628, -0.52059043, -0.60186252, -0.42716726,  2.15936337,
         -0.76078391, -0.65105585, -0.13506363, -2.29521727,  1.50811776],
        [ 1.7648389 , -1.04155708, -1.43912931, -0.804952  ,  0.39230544,
         -0.04600603,  2.20639813, -1.42770988,  0.00412754,  1.01395777],
        [-0.5381456 , -0.45457833, -1.48474235,  0.65566491,  0.190558  ,
          0.54755684,  0.49753518, -0.1505676 , -1.62248917, -1.54557073],
        [-1.13553163,  0.2522295 , -1.50273098, -0.06173152,  0.11635276,
          1.224847

In [365]:
i = 4
d0 = dataset[:,i:i+1]
d0

array([[1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [366]:
Alpha_Q = wake_sample(n_dz,d0,value_set,Phi,activation_type,bias)
Alpha_Q

{'z0': array([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 'z1': array([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1]]),
 'z2': array([[1],
        [0],
        [1],
        [1],
        [1],
        [1]]),
 'z3': array([[0],
        [0],
        [0]]),
 'z4': [[1]]}

In [367]:
Theta,Loss_P,G,parameter_set = sleep_update_delta(Theta,Alpha_Q,lr,n_dz,value_set,activation_type,bias)
Theta

[[0.15636175]
 [0.3550469 ]]
[[ 0.74577729]
 [-0.16460142]
 [ 0.36496014]
 [-0.09660865]
 [ 0.31582683]]
[[ 0.7539933 ]
 [ 1.09743588]
 [ 0.14360652]
 [ 0.46148075]
 [-0.35607646]
 [-0.29649025]
 [ 0.04985954]]
[[ 0.11138586]
 [ 0.48970154]
 [ 0.72092184]
 [-0.02659587]
 [-0.3115253 ]
 [-0.19046928]
 [-0.22422034]
 [-0.00498524]
 [ 0.18256938]]


{'Theta_10_1': array([[ 0.6028122 , -0.48238869,  0.68023287, -0.82012369,  0.93856391,
          2.29250678,  0.58267416,  0.7414749 ,  0.98462615, -1.59952184],
        [-0.81734877,  1.06284542,  0.98334406, -0.81202968, -0.73877867,
         -1.28313473, -0.69415029, -0.32941821, -1.03560493, -1.10131146],
        [-0.78710502, -1.2692291 , -0.49791989, -1.1606699 ,  0.45008676,
          0.29080188,  0.77710347,  1.23671295,  0.23177864,  0.40651893],
        [-1.34825199, -0.52243292, -0.60379978, -0.42804209,  2.15874809,
         -0.76215816, -0.6531228 , -0.13515768, -2.2961547 ,  1.50563078],
        [ 1.77377031, -1.02779452, -1.42465892, -0.79841745,  0.39690131,
         -0.03574104,  2.22183723, -1.42700739,  0.01112976,  1.0325344 ],
        [-0.51895583, -0.42500851, -1.45365171,  0.66970484,  0.20043255,
          0.5696119 ,  0.53070717, -0.14905825, -1.60744442, -1.50565755],
        [-1.11751265,  0.27999522, -1.47353722, -0.04854818,  0.12562485,
          1.245557

In [357]:
parameter_set

{'Theta_10_2': array([[ 0.00675541,  0.00675541,  0.00675541,  0.00675541,  0.00675541,
          0.        ,  0.        ,  0.00675541,  0.00675541],
        [ 0.00432536,  0.00432536,  0.00432536,  0.00432536,  0.00432536,
          0.        ,  0.        ,  0.00432536,  0.00432536],
        [ 0.00010898,  0.00010898,  0.00010898,  0.00010898,  0.00010898,
          0.        ,  0.        ,  0.00010898,  0.00010898],
        [ 0.01201875,  0.01201875,  0.01201875,  0.01201875,  0.01201875,
          0.        ,  0.        ,  0.01201875,  0.01201875],
        [ 0.01508518,  0.01508518,  0.01508518,  0.01508518,  0.01508518,
          0.        ,  0.        ,  0.01508518,  0.01508518],
        [ 0.02055841,  0.02055841,  0.02055841,  0.02055841,  0.02055841,
          0.        ,  0.        ,  0.02055841,  0.02055841],
        [-0.01633431, -0.01633431, -0.01633431, -0.01633431, -0.01633431,
         -0.        , -0.        , -0.01633431, -0.01633431],
        [ 0.00105825,  0.00105825,

In [358]:
G

{'z0': array([[1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1]]),
 'z1': array([[0.95980092],
        [0.38242383],
        [0.00098304],
        [0.5239688 ],
        [0.86631353],
        [0.57843195],
        [0.71782037],
        [0.02005306],
        [0.00939198]])}

In [ ]:


Alpha_P = sleep_sample(n_dz,value_set,Theta,activation_type,bias)
Phi,Loss_Q = wake_update_delta(Phi,Alpha_P,lr,n_dz,value_set,activation_type,bias)